In [1]:
import os

In [3]:
os.getcwd()

'C:\\Users\\devanandh.v'

In [4]:
os.chdir('D:\\Projects\\App\\Code')

In [5]:
os.getcwd()

'D:\\Projects\\App\\Code'

SyntaxError: invalid syntax (<ipython-input-9-ce48075e8c16>, line 1)